The main aim of the project is to implement Machine Translation using LSTM. LSTM Model is being considered in order to maintain Semantics of the Sentences. 

In [ ]:
##Basic Libraries for the Project
import numpy as np
from google.colab import files
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras import Model


In [ ]:
#File with the Translation Data
from google.colab import files
uploaded1 = files.upload()
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded1['final.csv']))

Saving final.csv to final (1).csv


In [ ]:
df.tail()

,English,Marathi
29995,do your parents know about this?,\tतुझ्या आईबाबांना याबद्दल माहीत आहे का?\n
29996,do your parents know about this?,\tतुमच्या आईवडिलांना याबद्दल माहीत आहे का?\n
29997,does anybody want anything else?,\tकोणाला अजून काही हवं आहे का?\n
29998,does he study english every day?,\tतो दररोज इंग्रजीचा अभ्यास करतो का?\n
29999,don't even think of going there.,\tतिथे जायचा विचारही करू नकोस.\n


In [ ]:
batch_size = 100  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 300  # Latent dimensionality of the encoding space.
num_samples = 20000 

input_texts = df.English
target_texts = df.Marathi
input_characters = set()
target_characters = set()


for line in range(num_samples):
    
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    
    
    for char in input_texts[line]:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_texts[line]:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

Number of samples: 30000
Number of unique input tokens: 53
Number of unique output tokens: 93
Max sequence length for inputs: 32
Max sequence length for outputs: 59


In [ ]:
input_characters = [' ','!','"','$','%',"'",',','-','.','/','0','1','2','3','4','5','6','7','8','9',':','?','A','B','C','D','E','F','G','H','I','J','K','L','M',
                    'N','O','P','Q','R','S','T','U','V','W','X','Y','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x',
                    'y','z','\xa0','’','€']

In [ ]:
num_encoder_tokens = len(input_characters)

In [ ]:
target_characters = ['\t','\n',' ','!','"','$','%',"'",'(',')',',','-','.','0','1','3',':','?','D','S','ँ','ं','ः','अ','आ','इ','ई','उ','ऊ','ऋ','ए','ऐ','ऑ','ओ',
                      'औ','क','ख','ग','घ','च','छ','ज','झ','ञ','ट','ठ','ड','ढ','ण','त','थ','द','ध','न','प','फ','ब','भ','म','य','र','ऱ','ल','ळ', 'व','श','ष',
                     'स','ह','़','ा','ि','ी','ु','ू','ृ','ॅ','े','ै','ॉ','ो','ौ','्','।','०','१','२','३','४','५','६','७','८','९','\u200d','€']

In [ ]:
num_decoder_tokens = len(target_characters)

The above step is done to ensure that no character is left and if any character occurs in the test set other than train it can be translated hence achieving more accuracy. 

In [ ]:
from tensorflow.keras.layers import Input
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
c=[]
count=0
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        count+=1
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    c.append(count)
    count=0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
encoder_input_data[1:2], input_texts[1] ,c[1] , input_token_index['!']

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)


Epoch 1/100
240/240 [==============================] - 5s 22ms/step - loss: 1.2360 - accuracy: 0.7094 - val_loss: 1.2733 - val_accuracy: 0.6731
Epoch 2/100
240/240 [==============================] - 5s 20ms/step - loss: 0.8010 - accuracy: 0.7842 - val_loss: 1.0694 - val_accuracy: 0.7073
Epoch 3/100
240/240 [==============================] - 5s 20ms/step - loss: 0.6876 - accuracy: 0.8073 - val_loss: 0.9635 - val_accuracy: 0.7333
Epoch 4/100
240/240 [==============================] - 5s 20ms/step - loss: 0.6133 - accuracy: 0.8279 - val_loss: 0.8964 - val_accuracy: 0.7548
Epoch 5/100
240/240 [==============================] - 5s 20ms/step - loss: 0.5583 - accuracy: 0.8434 - val_loss: 0.8380 - val_accuracy: 0.7717
Epoch 6/100
240/240 [==============================] - 5s 20ms/step - loss: 0.5182 - accuracy: 0.8542 - val_loss: 0.8004 - val_accuracy: 0.7810
Epoch 7/100
240/240 [==============================] - 5s 20ms/step - loss: 0.4858 - accuracy: 0.8628 - val_loss: 0.7644 - val_accuracy:

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(200,220):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print(input_seq.shape)

-
Input sentence: tom lies.
Decoded sentence: टॉम खोटं बोलतो.

(1, 32, 76)
-
Input sentence: tom lost.
Decoded sentence: टॉम हरला.

(1, 32, 76)
-
Input sentence: tom wept.
Decoded sentence: टॉम रडला.

(1, 32, 76)
-
Input sentence: tom's up.
Decoded sentence: टॉम उठला आहे.

(1, 32, 76)
-
Input sentence: use this.
Decoded sentence: हे वापर.

(1, 32, 76)
-
Input sentence: use this.
Decoded sentence: हे वापर.

(1, 32, 76)
-
Input sentence: warn tom.
Decoded sentence: टॉमला सांग.

(1, 32, 76)
-
Input sentence: warn tom.
Decoded sentence: टॉमला सांग.

(1, 32, 76)
-
Input sentence: watch me.
Decoded sentence: मला बघा.

(1, 32, 76)
-
Input sentence: watch me.
Decoded sentence: मला बघा.

(1, 32, 76)
-
Input sentence: watch us.
Decoded sentence: आम्हाला बघ.

(1, 32, 76)
-
Input sentence: watch us.
Decoded sentence: आम्हाला बघ.

(1, 32, 76)
-
Input sentence: we'll go.
Decoded sentence: आम्ही जाऊ.

(1, 32, 76)
-
Input sentence: we'll go.
Decoded sentence: आम्ही जाऊ.

(1, 32, 76)
-
Input sentence: 

In [ ]:
def Input_encoder(m):     
      encoder_input_data1 = np.zeros(
          (1, 32, 76),
          dtype='float32')
      
      
      for t, char in enumerate(m):
          #print(char)
          encoder_input_data1[0, t, input_token_index[char]] = 1.
      encoder_input_data1[0, t + 1:, input_token_index[' ']] = 1.

      return encoder_input_data1

m1 = input('Enter:')
x2 = Input_encoder(m1)
input_seq = x2
decoded_sentence = decode_sequence(input_seq)
print('-')
print('Input sentence:', m1)
print('Decoded sentence:', decoded_sentence)
print(input_seq.shape)

Enter:watch him
-
Input sentence: watch him
Decoded sentence: थांबव.

(1, 32, 76)
